# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_vac_df = pd.read_csv("../output_data/city_data1.csv")
city_vac_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,benito juarez,75,MX,1587428189,34,19.39,-99.17,80.60,17.22
1,1,hobart,40,AU,1587427898,66,-42.88,147.33,55.00,9.17
2,2,rikitea,24,PF,1587428092,71,-23.12,-134.97,77.13,9.42
3,3,santiago,69,CL,1587428102,23,-33.46,-70.65,77.00,8.05
4,4,victoria,20,HK,1587428114,74,22.29,114.16,81.00,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
vac_locations = city_vac_df[["Lat", "Lng"]]
vac_weight    = city_vac_df['Humidity'].astype(int)

# Plot Heatmap
gmaps_fig = gmaps.figure()

# Create heat layer
gmaps_heat = gmaps.heatmap_layer(vac_locations, weights=vac_weight,dissipating=False, max_intensity=10,point_radius=1) 
                                   
# Add layer
gmaps_fig.add_layer(gmaps_heat)
gmaps_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
fav_cities_df = city_vac_df.copy()
fav_cities_df = fav_cities_df[fav_cities_df['Max Temp'] > 65]
fav_cities_df = fav_cities_df[fav_cities_df['Max Temp'] < 85]  

fav_cities_df = fav_cities_df[fav_cities_df['Cloudiness'] < 50]
fav_cities_df = fav_cities_df[fav_cities_df['Humidity'] < 60]

fav_cities_df = fav_cities_df.dropna()

fav_cities_df = fav_cities_df.reset_index()
fav_cities_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,5,5,hilo,1,US,1587428132,54,19.73,-155.09,82.40,18.34
1,82,82,paithan,0,IN,1587428240,20,19.48,75.38,75.67,4.94
2,83,83,gazli,0,UZ,1587428241,25,40.13,63.45,65.86,2.28
3,120,120,beitbridge,0,ZW,1587428264,56,-22.22,30.00,67.21,6.69
4,134,134,tessalit,0,ML,1587428103,14,20.20,1.01,79.52,12.50
5,146,146,villa carlos paz,0,AR,1587428280,56,-31.42,-64.50,75.00,5.82
6,153,153,mungeli,28,IN,1587428289,27,22.07,81.68,80.08,0.74
7,169,169,flinders,40,AU,1587428311,55,-34.58,150.86,70.00,6.93
8,194,194,kahului,40,US,1587428160,45,20.89,-156.47,84.20,6.93
9,195,195,hun,0,LY,1587428329,24,29.13,15.95,75.06,8.16


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = fav_cities_df
hotel_df["Hotel Name"] = ""
hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,5,5,hilo,1,US,1587428132,54,19.73,-155.09,82.40,18.34,
1,82,82,paithan,0,IN,1587428240,20,19.48,75.38,75.67,4.94,
2,83,83,gazli,0,UZ,1587428241,25,40.13,63.45,65.86,2.28,
3,120,120,beitbridge,0,ZW,1587428264,56,-22.22,30.00,67.21,6.69,
4,134,134,tessalit,0,ML,1587428103,14,20.20,1.01,79.52,12.50,
5,146,146,villa carlos paz,0,AR,1587428280,56,-31.42,-64.50,75.00,5.82,
6,153,153,mungeli,28,IN,1587428289,27,22.07,81.68,80.08,0.74,
7,169,169,flinders,40,AU,1587428311,55,-34.58,150.86,70.00,6.93,
8,194,194,kahului,40,US,1587428160,45,20.89,-156.47,84.20,6.93,
9,195,195,hun,0,LY,1587428329,24,29.13,15.95,75.06,8.16,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map